In [20]:
# import libraries
from datetime import datetime, timedelta
import os
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division

import plotly.express as px

# import plotly.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go

#initiate visualization library for jupyter notebook 
# pyoff.init_notebook_mode()

In [9]:
tx_data = pd.read_csv(r'..\data\OnlineRetail.csv', encoding="ISO-8859-1")

## Data Informations

In [91]:
tx_data.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYearMonth,InvoiceYear,InvoiceMonth,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,201012,2010,12,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,2010,12,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,201012,2010,12,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,2010,12,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,2010,12,20.34


In [92]:
tx_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   InvoiceNo         541909 non-null  object        
 1   StockCode         541909 non-null  object        
 2   Description       540455 non-null  object        
 3   Quantity          541909 non-null  int64         
 4   InvoiceDate       541909 non-null  datetime64[ns]
 5   UnitPrice         541909 non-null  float64       
 6   CustomerID        406829 non-null  float64       
 7   Country           541909 non-null  object        
 8   InvoiceYearMonth  541909 non-null  int64         
 9   InvoiceYear       541909 non-null  int32         
 10  InvoiceMonth      541909 non-null  int32         
 11  Revenue           541909 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int32(2), int64(2), object(4)
memory usage: 45.5+ MB


In [93]:
tx_data.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID,InvoiceYearMonth,InvoiceYear,InvoiceMonth,Revenue
count,541909.000000,541909,541909.000000,406829.000000,541909.000000,541909.000000,541909.000000,541909.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570,201099.713989,2010.921609,7.553128,17.987795
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000,201012.000000,2010.000000,1.000000,-168469.600000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000,201103.000000,2011.000000,5.000000,3.400000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000,201107.000000,2011.000000,8.000000,9.750000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000,201110.000000,2011.000000,11.000000,17.400000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000,201112.000000,2011.000000,12.000000,168469.600000
std,218.081158,NaN,96.759853,1713.600303,25.788703,0.268787,3.509055,378.810824


Data limited to only one year of data : 2011

## Monthly Revenue
Revenue = Active Customer Count * Order Count * Average Revenue per Order

In [11]:
#converting the type of Invoice Date Field from string to datetime.
tx_data['InvoiceDate'] = pd.to_datetime(tx_data['InvoiceDate'])

#creating YearMonth field for the ease of reporting and visualization
tx_data['InvoiceYearMonth'] = tx_data['InvoiceDate'].map(lambda date: 100*date.year + date.month)
tx_data['InvoiceYear'] = tx_data['InvoiceDate'].dt.year
tx_data['InvoiceMonth'] = tx_data['InvoiceDate'].dt.month

#calculate Revenue for each row and create a new dataframe with YearMonth - Revenue columns
tx_data['Revenue'] = tx_data['UnitPrice'] * tx_data['Quantity']
tx_revenue = tx_data.groupby(['InvoiceYear','InvoiceMonth'])['Revenue'].sum().reset_index()
tx_revenue['InvoiceDate'] = pd.to_datetime(
    tx_revenue['InvoiceYear'].astype(str) + '-' + tx_revenue['InvoiceMonth'].astype(str) + '-01'
)
tx_revenue

,InvoiceYear,InvoiceMonth,Revenue,InvoiceDate
0,2010,12,748957.020,2010-12-01
1,2011,1,560000.260,2011-01-01
2,2011,2,498062.650,2011-02-01
3,2011,3,683267.080,2011-03-01
4,2011,4,493207.121,2011-04-01
5,2011,5,723333.510,2011-05-01
6,2011,6,691123.120,2011-06-01
7,2011,7,681300.111,2011-07-01
8,2011,8,682680.510,2011-08-01
9,2011,9,1019687.622,2011-09-01


In [30]:
# Assuming `tx_revenue` is a DataFrame with columns 'InvoiceYearMonth' and 'Revenue'
fig = px.line(
    tx_revenue,
    x='InvoiceDate',
    y='Revenue',
    title='Monthly Revenue',
    labels={'InvoiceDate': 'Year-month', 'Revenue': 'Revenue'}  # Optional: Customize axis labels
)

fig.update_traces(marker=dict(color="#1F77B4"))  # Classic blue
fig.update_layout(plot_bgcolor="white", template="plotly_white")

# Display the plot
fig.show()

In [33]:
#using pct_change() function to see monthly percentage change
tx_revenue['MonthlyGrowth'] = tx_revenue['Revenue'].pct_change()

#showing first 5 rows
tx_revenue.head()

#visualization - line graph
plot_data = [
    go.Scatter(
        x=tx_revenue.query("InvoiceDate < '2011-12-01'")['InvoiceDate'].dt.date,
        y=tx_revenue.query("InvoiceDate < '2011-12-01'")['MonthlyGrowth'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category",
               "tickformat":"%Y-%m"},
        title='Montly Growth Rate'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
fig.update_traces(marker=dict(color="#1F77B4"))  # Classic blue
fig.update_layout(plot_bgcolor="white", template="plotly_white")
fig.update_layout(xaxis=dict(tickangle=60))
pyoff.iplot(fig)

Observed month of negative and postive profits for 2011

## Monthly Customers
Number of different customer per month

In [87]:
tx_customers = tx_data.groupby(['InvoiceYear','InvoiceMonth','CustomerID']).count().reset_index()
tx_customers = tx_customers.groupby(['InvoiceYear','InvoiceMonth'])['CustomerID'].count().reset_index()
tx_customers['InvoiceDate'] = pd.to_datetime(
    tx_customers['InvoiceYear'].astype(str) + '-' + tx_customers['InvoiceMonth'].astype(str) + '-01'
)
tx_customers.head()

,InvoiceYear,InvoiceMonth,CustomerID,InvoiceDate
0,2010,12,948,2010-12-01
1,2011,1,783,2011-01-01
2,2011,2,798,2011-02-01
3,2011,3,1020,2011-03-01
4,2011,4,899,2011-04-01


In [100]:
# visualizing by bar chart
plot_data = [go.Bar(x=tx_customers['InvoiceDate'], 
                    y=tx_customers['CustomerID'])]

plot_layout = go.Layout(
        xaxis={#"type": "category",
               "tickformat":"%Y-%m"},
        title='Monthly Customers per Month'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
fig.update_traces(marker=dict(color="#1F77B4"))  # Classic blue
fig.update_layout(plot_bgcolor="white", template="plotly_white")
fig.update_layout(xaxis=dict(tickangle=60))
pyoff.iplot(fig)

Last quarter of 2011 indicates higher customers, potentially due to festive seasons

## Monthly Items Sold per month

In [101]:
tx_order = tx_data.groupby(['InvoiceYear','InvoiceMonth'])['Quantity'].sum().reset_index()
tx_order['InvoiceDate'] = pd.to_datetime(
    tx_order['InvoiceYear'].astype(str) + '-' + tx_order['InvoiceMonth'].astype(str) + '-01')
tx_order.head()

,InvoiceYear,InvoiceMonth,Quantity,InvoiceDate
0,2010,12,342228,2010-12-01
1,2011,1,308966,2011-01-01
2,2011,2,277989,2011-02-01
3,2011,3,351872,2011-03-01
4,2011,4,289098,2011-04-01


In [102]:
# visualizing by bar chart
plot_data = [go.Bar(x=tx_order['InvoiceDate'], 
                    y=tx_order['Quantity'])]

plot_layout = go.Layout(
        xaxis={#"type": "category",
               "tickformat":"%Y-%m"},
        title='Monthly Items Sold per Month'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
fig.update_traces(marker=dict(color="#1F77B4"))  # Classic blue
fig.update_layout(plot_bgcolor="white", template="plotly_white")
fig.update_layout(xaxis=dict(tickangle=60))
pyoff.iplot(fig)

## New Customers per month

In [103]:
tx_data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYearMonth,InvoiceYear,InvoiceMonth,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,201012,2010,12,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,2010,12,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,201012,2010,12,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,2010,12,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,2010,12,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,201112,2011,12,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,201112,2011,12,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,201112,2011,12,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,201112,2011,12,16.60


In [125]:
tx_ncustomers = tx_data.groupby(['InvoiceYear','InvoiceMonth'])['CustomerID'].unique().reset_index()
tx_ncustomers['InvoiceDate'] = pd.to_datetime(
    tx_ncustomers['InvoiceYear'].astype(str) + '-' + tx_ncustomers['InvoiceMonth'].astype(str) + '-01')
tx_ncustomers.head()


,InvoiceYear,InvoiceMonth,CustomerID,InvoiceDate
0,2010,12,"[17850.0, 13047.0, 12583.0, 13748.0, 15100.0, ...",2010-12-01
1,2011,1,"[13313.0, nan, 18097.0, 16656.0, 16875.0, 1309...",2011-01-01
2,2011,2,"[15240.0, 14911.0, 14496.0, 17147.0, 17675.0, ...",2011-02-01
3,2011,3,"[14620.0, 14740.0, 13880.0, 16462.0, 17068.0, ...",2011-03-01
4,2011,4,"[18161.0, 14886.0, 17613.0, 12523.0, 13694.0, ...",2011-04-01


In [131]:
# tx_customers.loc[0,['CustomerID']].values[0]
date_list = tx_ncustomers.InvoiceDate.sort_values()
for i, date_ in enumerate(date_list):
    temp_ = tx_customers.loc[i,['CustomerID']].values[0]
    if date_ == date_list[0] or (date_ == date_list[-1]):
        temp_before = []
    else:
        temp_before = tx_customers.loc[i+1,['CustomerID']].values[0]
    list_id = set(temp_) - set(temp_before)
    

KeyError: 13

In [130]:
tx_customers.loc[0,['CustomerID']].values[0]

array([17850., 13047., 12583., 13748., 15100., 15291., 14688., 17809.,
       15311., 14527., 16098., 18074., 17420., 16029., 16250., 12431.,
       17511., 17548., 13705., 13747., 13408., 13767., 17924., 13448.,
       15862., 15513., 12791., 16218., 14045., 14307., 17908., 17920.,
          nan, 12838., 13255., 16583., 18085., 13758., 13694., 15983.,
       14849., 17968., 16210., 17897., 17377., 16552., 17181., 17951.,
       14729., 12748., 15012., 12868., 17572., 14078., 14001., 12662.,
       15525., 14237., 17905., 15485., 12433., 16955., 15350., 15605.,
       18144., 15922., 14594., 15165., 14911., 16456., 17841., 12472.,
       17346., 17643., 17873., 13093., 12921., 13468., 17760., 16928.,
       16048., 16274., 14496., 14696., 16539., 17025., 13777., 17690.,
       12947., 17460., 18229., 14142., 17069., 13065., 14606., 16835.,
       15235., 13576., 18011., 13090., 15694., 14741., 13715., 14092.,
       17732., 12855., 15752., 17855., 14047., 17925., 13941., 17017.,
      